# Imports

In [53]:
import os
import pandas as pd
import keras
import tensorflow as tf

# Modify sys.path 
project_root = '/Users/carlesferreres/Desktop/Carles/Empresas/KOA/Repos/aquagen-experimentation/'
os.chdir(project_root)

# Config

In [54]:
# Model config
exp_id = 'acc1'
model_name = 'nn_model_acc1_20240512.keras'
dir = f'data/input_data/exp_{exp_id}'

retrain = False

# Load data and model

In [55]:
# Load data
X_test = pd.read_csv(os.path.join(dir, 'X_test.csv'))
y_test = pd.read_csv(os.path.join(dir, 'y_test.csv'))['Class']

In [56]:
# Read model
model = keras.models.load_model(os.path.join('models', model_name))

In [57]:
if retrain:
    X_train = pd.read_csv(os.path.join(dir, 'X_train.csv'))
    y_train = pd.read_csv(os.path.join(dir, 'y_train.csv'))['Class']
    model.fit(X_train, y_train, epochs=200, validation_split=0.2, verbose=1)

# Model evaluation

## Metrics

In [58]:
# Test set distribution
y_test.value_counts()

Class
True     169
False     44
Name: count, dtype: int64

In [59]:
# Make predictions and evaluate main metrics
y_pred = model.predict(X_test)
auc_prc = keras.metrics.AUC(curve='PR', name='auc_prc')(y_test, y_pred) 
roc_auc = keras.metrics.AUC(name='roc_auc')(y_test, y_pred) 

print("AUC-PRC:", auc_prc.numpy())
print("ROC-AUC:", roc_auc.numpy())

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
AUC-PRC: 0.9518323
ROC-AUC: 0.8938946


In [60]:
# Calculate confusion matrix
threshold = 0.5
y_pred_rounded = tf.cast(tf.greater(y_pred, threshold), tf.float32)

cm = tf.math.confusion_matrix(y_test, y_pred_rounded, num_classes=2)
cm_df = pd.DataFrame(cm,
                     columns=['false', 'true'],
                     index=['false', 'true'])
cm_df.columns = pd.MultiIndex.from_product([['predicted'], cm_df.columns])
cm_df.index = pd.MultiIndex.from_product([['actual'], cm_df.index])
cm_df

predicted     
                 false true
actual false        34   10
       true          8  161

In [61]:
# Extract tn, fp, fn, tp values:
TN = cm[0, 0].numpy()
FP = cm[0, 1].numpy()
FN = cm[1, 0].numpy()
TP = cm[1, 1].numpy()

In [62]:
# Calculate test metrics
acc = 100*(TP+TN)/(TP+TN+FP+FN)
precision = 100*TP/(TP+FP)
recall = 100*TP/(TP+FN)
fallout = 100*FP/(TN+FP)
_for = 100*FN/(TN+FN)
tnr = 100 - _for
balanced_acc = (tnr + recall)/2
f1 = 2*(precision*recall)/(precision+recall)

## Insights

In [63]:
print(f'The overall model accuracy is {acc:.2f}%')
print(f'The overall model balanced accuracy is {balanced_acc:.2f}%')

The overall model accuracy is 91.55%
The overall model balanced accuracy is 88.11%


In [64]:
print(f'Given that there is a pathogen in the sample, the model is {recall:.2f}% likely to detect it.')
print(f'Given that there is no pathogen in the sample, the model is {fallout:.2f}% likely to wrongly detect it.')

Given that there is a pathogen in the sample, the model is 95.27% likely to detect it.
Given that there is no pathogen in the sample, the model is 22.73% likely to wrongly detect it.


In [65]:
print(f'Given that the model gave a positive result (pathogen), the sample is {precision:.2f}% likely to have a pathogen.')
print(f'Given that the model gave a negative result (no pathogen), the sample is still {_for:.2f}% likely to have a pathogen.')

Given that the model gave a positive result (pathogen), the sample is 94.15% likely to have a pathogen.
Given that the model gave a negative result (no pathogen), the sample is still 19.05% likely to have a pathogen.
